In [1]:
import cv2
import pandas as pd
import numpy as np
from mediapipe.python.solutions import pose as mp_pose
from sklearn.preprocessing import StandardScaler
import os
from glob import glob

In [2]:
# 1. ポーズ抽出関数（x, y, zのみ）
def extract_pose_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose(static_image_mode=False)
    all_landmarks = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            frame_data = []
            for lm in results.pose_landmarks.landmark:
                frame_data.extend([lm.x, lm.y, lm.z])
            all_landmarks.append(frame_data)

    cap.release()

    if all_landmarks:
        columns = [f'P{i}_{axis}' for i in range(33) for axis in ('x', 'y', 'z')]
        df = pd.DataFrame(all_landmarks, columns=columns)
        return df
    else:
        print(f"⚠️ No landmarks extracted from {video_path}")
        return None

# 2. 前処理関数（補完・スケーリング・フレーム数統一）
def preprocess_pose_dataframe(df):
    df = df.copy()

    # 欠損補完
    df.interpolate(method='linear', limit_direction='both', inplace=True)
    df.fillna(df.mean(), inplace=True)

    # # スケーリング
    # scaler = StandardScaler()
    # df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

    return df


In [3]:
# --- パス設定 ---
input_dir = "../data/動画データ_ver3"
output_dir = "../data/normalized_features/0814"
os.makedirs(output_dir, exist_ok=True)

# --- .mp4 と .mov の両方を対象に ---
video_paths = sorted(
    glob(os.path.join(input_dir, "*.mp4")) +
    glob(os.path.join(input_dir, "*.mov"))
)

print(f"🎬 対象動画ファイル数: {len(video_paths)}")

for video_path in video_paths:
    filename = os.path.splitext(os.path.basename(video_path))[0]
    output_csv_path = os.path.join(output_dir, f"{filename}.csv")

    df_raw = extract_pose_from_video(video_path)
    if df_raw is not None:
        df_processed = preprocess_pose_dataframe(df_raw)
        df_processed.to_csv(output_csv_path, index=False)
        print(f"✅ 完了: {output_csv_path}")
    else:
        print(f"❌ スキップ: {video_path}")

🎬 対象動画ファイル数: 35
✅ 完了: ../data/normalized_features/0814\久世.csv
✅ 完了: ../data/normalized_features/0814\十河.csv
✅ 完了: ../data/normalized_features/0814\十河1人目リラックスVID20250616115433.csv
✅ 完了: ../data/normalized_features/0814\十河1人目嬉しいVID20250616115335.csv
✅ 完了: ../data/normalized_features/0814\十河1人目悲しいVID20250616115353.csv
✅ 完了: ../data/normalized_features/0814\十河1人目緊張VID20250616115419.csv
✅ 完了: ../data/normalized_features/0814\十河2人目リラックスVID20250616120023.csv
✅ 完了: ../data/normalized_features/0814\十河2人目嬉しいVID20250616115944.csv
✅ 完了: ../data/normalized_features/0814\十河2人目悲しいVID20250616115957.csv
✅ 完了: ../data/normalized_features/0814\十河2人目緊張VID20250616120010.csv
✅ 完了: ../data/normalized_features/0814\十河3人目リラックスVID20250616120127.csv
✅ 完了: ../data/normalized_features/0814\十河3人目嬉しいVID20250616120042.csv
✅ 完了: ../data/normalized_features/0814\十河3人目悲しいVID20250616120100.csv
✅ 完了: ../data/normalized_features/0814\十河3人目緊張VID20250616120112.csv
✅ 完了: ../data/normalized_features/0814\十河4人目リラックスVID202506161